<a href="https://colab.research.google.com/github/ananthgv-usk/swamiji-tts/blob/main/Sesame_TTS_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.52.3
!pip install --no-deps trl==0.22.2
!pip install torchcodec "datasets>=3.4.1,<4.0.0"
!pip install soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.2
    Uninstalling tokenizers-0.22.2:
      Successfully uninstalled tokenizers-0.22.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 50.4 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's de

In [4]:
from transformers import CsmForConditionalGeneration, AutoProcessor
import torch
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

model = CsmForConditionalGeneration.from_pretrained(
    "kailasa-ngpt/csm-1b-lora-2025-12-23",
    torch_dtype=torch.float16,
    device_map="cuda",
    token=hf_token
)

# Load processor from BASE model
processor = AutoProcessor.from_pretrained("unsloth/csm-1b", token=hf_token)

print("Ready!")

adapter_config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/116M [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Ready!


In [6]:
from datasets import load_dataset, Audio as DsAudio
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

# ============ LOAD DATASET ============
ds = load_dataset("kailasa-ngpt/SPH_Audio_2019_60_Secs_947_Samples", split="train", token=hf_token)
ds = ds.cast_column("audio", DsAudio(sampling_rate=24000))

# Check what columns exist
print(ds.column_names)  # e.g., ['audio', 'text', 'speaker_id']

# ============ GET REFERENCE FROM DATASET ============
sample_idx = 70  # Pick any sample

reference_audio = ds[sample_idx]["audio"]["array"]  # Audio numpy array
reference_text = ds[sample_idx]["text"]             # Transcript already in dataset!

print(f"Reference text: {reference_text}")

README.md:   0%|          | 0.00/390 [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/297M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/265M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/286M [00:00<?, ?B/s]

train-00003-of-00006.parquet:   0%|          | 0.00/275M [00:00<?, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 5d3a9448-f152-4b53-bb50-28b6f52f96a7)')' thrown while requesting HEAD https://huggingface.co/datasets/kailasa-ngpt/SPH_Audio_2019_60_Secs_947_Samples/resolve/0d83e99d038792f0fe58486d00f6b5477e4ed5f9/data/train-00004-of-00006.parquet
Retrying in 1s [Retry 1/5].


train-00004-of-00006.parquet:   0%|          | 0.00/293M [00:00<?, ?B/s]

train-00005-of-00006.parquet:   0%|          | 0.00/300M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/947 [00:00<?, ? examples/s]

['audio', 'text', 'source']
Reference text: People who go on spiritually killing you by constantly putting abuses, destructive, negative ideas into you are demons. Please be very clear, Kalabhairava is taking care of demons, devils, Rakshasas. He is destroying them. Kalabhairava and Mahakali are destroying demons, devils, Rakshasas, Rakshasis. So you don't worry about them. Just do not allow them to poison you and spiritually kill you. Do not get killed spiritually by abusers, demons, devils, rakshasas, rakshasis. Be very Rakshasis. Be very clear about that.


In [7]:
import torch
import soundfile as sf
from IPython.display import Audio

text_to_speak = "Listen intensely, Anant. I understand your deep yearning to understand the Arjuna Krishna relationship. That sacred disciple-guru dynamic you've been seeking to embody with me."
speaker_id = 0

# Convert the reference_audio numpy array to a torch tensor with float16 dtype
# The processor expects a numpy array, so we convert it back after setting the dtype.
reference_audio_for_processor = torch.tensor(reference_audio, dtype=torch.float16).cpu().numpy()

conversation = [
    {
        "role": str(speaker_id),
        "content": [
            {"type": "text", "text": reference_text},  # From dataset
            {"type": "audio", "path": reference_audio_for_processor} # Use the float16 numpy array
        ]
    },
    {
        "role": str(speaker_id),
        "content": [
            {"type": "text", "text": text_to_speak}
        ]
    },
]

inputs = processor.apply_chat_template(
    conversation, tokenize=True, return_dict=True,
)

# Move inputs to cuda and ensure float tensors are float16
inputs_on_device = {}
for k, v in inputs.items():
    if isinstance(v, torch.Tensor):
        if v.dtype == torch.float32: # Only convert float32 tensors to float16
            inputs_on_device[k] = v.to("cuda", dtype=torch.float16)
        else: # For other types like long, int, bool, just move to cuda
            inputs_on_device[k] = v.to("cuda")
    else:
        inputs_on_device[k] = v

audio_values = model.generate(
    **inputs_on_device,
    max_new_tokens=1500,
    output_audio=True,
)

audio = audio_values[0].to(torch.float32).cpu().numpy()
sf.write("output.wav", audio, 24000)
display(Audio(audio, rate=24000))